# Case Study 1 : Collecting Data from Twitter

Due Date: September 21, **BEFORE the beginning of class at 6:00pm**

## **NOTE: There are *always* last minute issues submitting the case studies.  DO NOT WAIT UNTIL THE LAST MINUTE!**

* ------------

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    Yao Chun Hsieh
    Nathan Hsu
    Yuchen Shen
    Yang Tao
    Ying Fang

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [2]:
import twitter
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'ScmFh6jc3YxqwcNxCQrKon3nS'
    CONSUMER_SECRET = 'GK01LyBIQFq4pM3UdYXashxTKfWRNCtEnpjDdlJLLVoM1PrCsc'
    OAUTH_TOKEN = '906960184504979457-vvdrt7JqA2OBtXCbcw4theY6wbaOirJ'
    OAUTH_TOKEN_SECRET = 'kgQQ0G46MoAntkc0PGEplf6a1MkyRsw83gbDHs1cTFlyU'

    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api


In [5]:
# Import unquote to prevent url encoding errors in next_results
from urllib.parse import quote, unquote
from datetime import datetime

# Query Setting
keyWord = 'SteveJobs Theater'
count = 100

twitter_api = oauth_login()
search_results = twitter_api.search.tweets(q=keyWord, count=count)
statuses = search_results['statuses']

# Iterate through 10 more batches of results by following the cursor
for _ in range(5):
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

import json
print("Length of statuses", len(statuses))
data = json.dumps(statuses, indent=4)
    
curTime = datetime.now().strftime('%y%m%d_%H%M%S')
fileDirectory = "%s-%s(%s).txt" % (keyWord, curTime, str(len(statuses)))
    
file = open(fileDirectory,'w')
file.write(data)
file.close()
print("Data Saved: " + fileDirectory)

Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 500
Length of statuses 508
Data Saved: SteveJobs Theater-170919_223450(508).txt


##### Report some statistics about the tweets you collected 

* The topic of interest: SteveJobs Theater


* The total number of tweets collected:  508

*-----------------------

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

Load file from disk

In [7]:
with open("SteveJobs Theater-170919_223450(508).txt") as tweetfile:
    statuses = json.load(tweetfile)

In [8]:
import time
status_texts = [status['text']
                for status in statuses]

# Compute a collection of all words from all tweets
words = [w
         for t in status_texts
             for w in t.split()]

screen_names = [user_mention['screen_name']
                for status in statuses
                    for user_mention in status['entities']['user_mentions']]

hashtags = [hashtag['text']
            for status in statuses
                for hashtag in status['entities']['hashtags']]


In [9]:
'''For future, we should remove meaningless word like 'a', 'the', 'an', 'is'... etc.
   and, we should see a word in both Upper/Lower case as the same word'''

from collections import Counter
from prettytable import PrettyTable

pt = PrettyTable(field_names=['Word', 'Count']) 
c = Counter(words)
[pt.add_row(kv) for kv in c.most_common()[:30]]
pt.align['Word'], pt.align['Count'] = 'l', 'r' # Set column alignment
print(pt)

+-------------------------+-------+
| Word                    | Count |
+-------------------------+-------+
| RT                      |   326 |
| #AppleEvent             |   247 |
| SteveJobs               |   208 |
| #SteveJobs              |   206 |
| the                     |   186 |
| Theater                 |   183 |
| Theaterの中です！       |   162 |
| https://t.co/FH0DmygoBu |   162 |
| @gizmodojapan:          |   161 |
| at                      |   125 |
| #Apple's                |   101 |
| for                     |    97 |
| #iPhoneX                |    90 |
| #iPhone8                |    71 |
| &amp;                   |    68 |
| @appleinsider:          |    67 |
| live                    |    64 |
| unveiling               |    62 |
| Steve                   |    60 |
| event                   |    60 |
| Join                    |    60 |
| Jobs                    |    58 |
| @AppleInsider           |    56 |
| https://t.co/0mo6UQHrbp |    56 |
| #stevejobs              |    55

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [10]:
retweets = []
for status in statuses:
    L = []
    for i in retweets:
        L.append(i[2])
    if status['text'] in L:
        continue
    if 'retweeted_status' in status:
        retweets.append(
             (status['retweet_count'], 
             status['retweeted_status']['user']['screen_name'],
             status['text']))      
           

# Slice off the first 5 from the sorted results and display each item in the tuple

pt = PrettyTable(field_names=['Count', 'Screen Name', 'Text'])
[pt.add_row(row) for row in sorted(retweets, reverse=True)[:10]]
pt.max_width['Text'] = 50
pt.align= 'l'
print(pt)

+-------+--------------+----------------------------------------------------+
| Count | Screen Name  | Text                                               |
+-------+--------------+----------------------------------------------------+
| 208   | gizmodojapan | RT @gizmodojapan: SteveJobs Theaterの中です！      |
|       |              | #AppleEvent https://t.co/FH0DmygoBu                |
| 62    | appleinsider | RT @appleinsider: Join @AppleInsider live at the   |
|       |              | #SteveJobs Theater for #Apple's #iPhoneX &amp;     |
|       |              | #iPhone8 unveiling https://t.co/0mo6UQHrbp http…   |
| 51    | appleinsider | RT @appleinsider: Latest #ApplePark drone video    |
|       |              | shows completed #SteveJobs Theater ahead of Sept.  |
|       |              | 12 event https://t.co/2bnK1ad8if https://…         |
| 21    | yuuuuuiiiii  | RT @yuuuuuiiiii: SteveJobs                         |
|       |              | TheaterはSFのワンシーンのようだ。音楽もいい。でも、ここには何にもないの。 |


**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [11]:
'''We should remove meaningless word like 'a', 'the', 'an', 'is'... etc.'''
'''We should see a word in both Upper/Lower case as the same word'''
from collections import Counter
from prettytable import PrettyTable

for label, data in (('Hashtag', hashtags),
                    ('Screen Name', screen_names)
                   ):
    pt = PrettyTable(field_names=[label, 'Count']) 
    c = Counter(data)
    [pt.add_row(kv) for kv in c.most_common()[:10]]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print(pt)

+------------+-------+
| Hashtag    | Count |
+------------+-------+
| AppleEvent |   257 |
| SteveJobs  |   212 |
| Apple      |   136 |
| iPhoneX    |    92 |
| iPhone8    |    72 |
| stevejobs  |    56 |
| apple      |    34 |
| iPhone     |    26 |
| iphone8    |    23 |
| Theater    |    19 |
+------------+-------+
+-----------------+-------+
| Screen Name     | Count |
+-----------------+-------+
| gizmodojapan    |   161 |
| appleinsider    |   123 |
| yuuuuuiiiii     |    36 |
| Apple           |    30 |
| CityofCupertino |    12 |
| cpassariello    |     7 |
| tim_cook        |     6 |
| arstechnica     |     4 |
| IntuzHQ         |     4 |
| DHQLounge       |     4 |
+-----------------+-------+


* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [ ]:
targetScreenName = "pricechopper"

#Chapter 9 Solution for All friends and follwers

import sys
import time
from urllib.request import URLError
from http.client import BadStatusLine
import json
import twitter

def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw): 
    
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
    
        if wait_period > 3600: # Seconds
            raise e
    
        # See https://dev.twitter.com/docs/error-codes-responses for common codes  
        if e.e.code == 401:
            return None
        elif e.e.code == 404:
            return None
        elif e.e.code == 429:
            if sleep_when_rate_limited:
                time.sleep(60*15 + 5)
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function
    
    wait_period = 2 
    error_count = 0 

    while True:
        try:
            return twitter_api_func(*args, **kw)
        except twitter.api.TwitterHTTPError as e:
            error_count = 0 
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError as e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            if error_count > max_errors:
                raise
        except BadStatusLine as e:
            error_count += 1
            time.sleep(wait_period)
            wait_period *= 1.5
            if error_count > max_errors:
                raise

# Sample usage
# See https://dev.twitter.com/docs/api/1.1/get/users/lookup for 
# twitter_api.users.lookup
#twitter_api = oauth_login()
#response = make_twitter_request(twitter_api.users.lookup, screen_name="tim_cook")


#Chapter 9 Solution for All friends and follwers
#retrieve every result and write into .txt immediately
from functools import partial
import tabulate
def get_friends_followers_info(twitter_api, screen_name=None, user_id=None):
    
    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"
    
    # See https://dev.twitter.com/docs/api/1.1/get/friends/ids and
    # https://dev.twitter.com/docs/api/1.1/get/followers/ids for details
    # on API parameters
    
    get_friends_list = partial(make_twitter_request, twitter_api.friends.list, count=200)
    get_followers_list = partial(make_twitter_request, twitter_api.followers.list, count=200)

    friends_info, followers_info = [], []
    
    for twitter_api_func, info, label in [
                    [get_friends_list, friends_info, "friends"], 
                    [get_followers_list, followers_info, "followers"]
                ]:
        
        cursor = -1
        while cursor !=0:
        
            # Use make_twitter_request via the partially bound callable...
            if screen_name: 
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)

            if response is not None:
                for user in response['users']:
                    info.append((user['id'], user['screen_name']))
                    file = open(label+'.txt','a')
                    file.write(str(info[-1])+'\n')
                    file.close()
                cursor = response['next_cursor']
                
        
            # XXX: You may want to store data during each iteration to provide an 
            # an additional layer of protection from exceptional circumstances     
            if response is None:
                break
        file = open(label+'.txt','a')
        file.write(str(len(info))+'\n')
        file.close()
    # Do something useful with the IDs, like store them to disk...
    return friends_info, followers_info
twitter_api = oauth_login()
friends_info, followers_info = get_friends_followers_info(twitter_api, screen_name = targetScreenName )

In [ ]:
#print 20 of friends and followers
f= open('followers.txt', 'r')
followers_info=[]
for line in f:
    values = line.split()
    followers_info.append((values[0][1:-1],values[1][1:-2]))
f.close()
f= open('friends.txt', 'r')
friends_info=[]
for line in f:
    values = line.split()
    friends_info.append((values[0][1:-1],values[1][1:-2]))
f.close()
count = 20
print("List %d Friend/Followers of %s: " % (count, targetScreenName))
from collections import Counter
from prettytable import PrettyTable
for label, data in (('Friends', friends_info),
                    ('Followers', followers_info)
                   ):
    pt = PrettyTable(field_names=['ID', label]) 
    [pt.add_row(row) for row in data[:count]]
    pt.align['ID'], pt.align[label] = 'l', 'l' # Set column alignment
    print(pt)
    print("\n")

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [12]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
    
#print 20 mutual
f= open('followers.txt', 'r')
followers_info=[]
for line in f:
    values = line.split()
    followers_info.append((values[0][1:-1],values[1][1:-2]))
f.close()
f= open('friends.txt', 'r')
friends_info=[]
for line in f:
    values = line.split()
    friends_info.append((values[0][1:-1],values[1][1:-2]))
f.close()
mutuals = list(set(friends_info).intersection(followers_info))

from prettytable import PrettyTable
pt = PrettyTable(field_names=['ID', 'Mutual Friends']) 
[pt.add_row(row) for row in mutuals[:count]]
pt.align['ID'], pt.align['Mutual Friends'] = 'l', 'l' # Set column alignment
print(pt)
print("\n")

NameError: name 'friends_info' is not defined

*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [1]:
import twitter
from prettytable import PrettyTable
import sys
import time
from urllib.request import URLError
from http.client import BadStatusLine
import json
import twitter
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'ScmFh6jc3YxqwcNxCQrKon3nS'
    CONSUMER_SECRET = 'GK01LyBIQFq4pM3UdYXashxTKfWRNCtEnpjDdlJLLVoM1PrCsc'
    OAUTH_TOKEN = '906960184504979457-vvdrt7JqA2OBtXCbcw4theY6wbaOirJ'
    OAUTH_TOKEN_SECRET = 'kgQQ0G46MoAntkc0PGEplf6a1MkyRsw83gbDHs1cTFlyU'

    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

In [49]:
import os
allFiles = []
def getFileName(path):
    ''' 获取指定目录下的所有指定后缀的文件名 '''

    f_list = os.listdir(path)
    # print f_list
    for i in f_list:
        # os.path.splitext():分离文件名与扩展名
        if os.path.splitext(i)[1] == '.txt':
            allFiles.append(i)
getFileName('/Users/shenyuchen/Desktop/NoCountAttheEnd/iPhone')

In [6]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

#Find most popular twitter at a exactly time, and find the most popular people, 
#locate the people who has the highest business value

#find top 15 retweet, 5 top follower 
file = open('output','w')
output = []
for F in allFiles:
    with open(F) as tweetfile:
        statuses = json.load(tweetfile)

    retweets = []
    for status in statuses:
        L = []
        for i in retweets:
            L.append(i[3])
        if status['text'] in L:
            continue
        if 'retweeted_status' in status:
            retweets.append(
                 (status['retweet_count'], 
                  status['retweeted_status']['user']['followers_count'], 
                  status['retweeted_status']['user']['screen_name'],
                  status['text']))      


    # Slice off the first 5 from the sorted results and display each item in the tuple
    ScrName = []
    for row in sorted(retweets, reverse=True)[:15]:
        ScrName.append((row[1],row[2],row[0]))

    pt = PrettyTable(field_names=['Count','followers_count', 'Screen Name',  'Text'])
    [pt.add_row(row) for row in sorted(retweets, reverse=True)[:15]]
    pt.max_width['Text'] = 50
    pt.align= 'l'
    output = (F,sorted(ScrName,reverse = True)[0:5])
    file.write('\n'+str(output)+'\t\n')


print("Data Saved: output")

#print(ScrName)
#print(pt)

Data Saved: output170913_154922iphoneX.txt
[(10091654, 'JohnCena', 267219), (396409, 'TopAchat', 52357), (697, 'nakurubox', 23732), (6367708, 'IGN', 18472), (53, 'Cerberus_Ignite', 16795), (12601, '_Do_t', 14361), (335885, 'ceemeagain', 9160), (956948, 'priwpriww', 7733), (47, 'biggie_teethara', 7633), (192, 'cowboy751', 7236), (997, 'its_kachi', 7202), (2931, 'ElijahMiano', 7002), (12600, '_Do_t', 6910), (811405, 'KTHopkins', 6381), (94780, 'SuperSaf', 6149)]
+--------+-----------------+-----------------+----------------------------------------------------+
| Count  | followers_count | Screen Name     | Text                                               |
+--------+-----------------+-----------------+----------------------------------------------------+
| 267219 | 10091654        | JohnCena        | RT @JohnCena: Sooo #iPhoneX about #FaceID          |
|        |                 |                 | ...ummmmm .... what do I do?                       |
| 52357  | 396409          | TopAch

*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA Wen Liu (wliu3@wpi.edu).

#### We auto-process the submissions so make sure your subject line is *exactly*:

### DS501 Case Study 1 Team ??

#### where ?? is your team number.
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

